In [1]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ


## Change it if needed!
os.environ["DIPY_HOME"] = "/home/shakeri/Desktop/MyFiles/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

import requests
import urllib.request
from tqdm import tqdm

# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

In [2]:
# Start FSLeyes within Python 
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

# Display
fsleyesDisplay.load('T1w.nii.gz')

Gtk-Message: 08:49:32.247: Failed to load module "canberra-gtk-module"
08:49:32: Debug: Adding duplicate image handler for 'Windows bitmap file'
08:49:32: Debug: Adding duplicate animation handler for '1' type
08:49:32: Debug: Adding duplicate animation handler for '2' type
08:49:32: Debug: Adding duplicate image handler for 'Windows bitmap file'
08:49:32: Debug: Adding duplicate animation handler for '1' type
08:49:32: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:5390): Gtk-CRITICAL **: 08:49:32.450: gtk_window_resize: assertion 'height > 0' failed


In [3]:
###################
# Skull Stripping #
###################

import os

def skull_strip_single_t1(t1_path, robust=False):
    """
    Skull-strip a single T1w image using FSL BET.
    Saves *_brain.nii.gz and *_mask.nii.gz next to the input file.
    """
    t1_dir = os.path.dirname(t1_path)
    t1_base = os.path.splitext(os.path.splitext(os.path.basename(t1_path))[0])[0]  # removes .nii.gz
    output_path = os.path.join(t1_dir, t1_base + "_brain")

    cmd = f"bet {t1_path} {output_path} -m {'-R' if robust else ''}"
    print(f"Running: {cmd}")
    os.system(cmd)
    print("✅ Skull stripping complete!")

# Call it on your file
skull_strip_single_t1('T1w.nii.gz')

Running: bet T1w.nii.gz T1w_brain -m 
Skull stripping complete!


In [10]:
# Display
fsleyesDisplay.load('T1w_brain.nii.gz')
fsleyesDisplay.load('T1w_brain_mask.nii.gz')

In [5]:
def apply_python_mask_approach(img_path, mask_path, masked_img_path):
    """
    Parameters
    ----------
    img_path: str
        Path to the image on which we would like to apply the mask (in your case, the T1 with the skull still on). Should be a .nii.gz file
    mask_path: str
        Path to the mask you would like to apply to your image. Should be a .nii.gz file, containing only binary values (0 or 1)
    masked_img_path: str
        Path to which the resulting image will be saved.
    """
    import nibabel as nib

    # Load both the T1 and the mask from disk
    img = nib.load(img_path)
    mask = nib.load(mask_path)
    
    # Load the data from both above images as numpy arrays
    img_data = img.get_fdata()
    mask_data = mask.get_fdata()

    # Create an empty image and select all which falls in the mask
    # In all positions within the mask, get the image content
    saved_img_data = np.zeros(img_data.shape)
    saved_img_data[mask_data > 0] = img_data[mask_data > 0]

    # Save the image to disk, by creating a new Nifti image and then writing it out
    img_out = nib.Nifti1Image(saved_img_data,img.affine, img.header)
    nib.save(img_out, masked_img_path)

def apply_fsl_math_approach(img_path, mask_path, masked_img_path):
    os.system('fslmaths {} -mas {} {}'.format(img_path, mask_path, masked_img_path))

In [6]:
import os

data_dir = os.getcwd()  # Change paths for your own data if needed!

# Input and output paths
anatomical_path = os.path.join(data_dir, 'T1w.nii.gz')                  # Original brain (with skull)
betted_brain_path = os.path.join(data_dir, 'T1w_brain.nii.gz')          # Skull-stripped brain
resulting_mask_path = os.path.join(data_dir, 'T1w_brain_mask.nii.gz')   # Brain mask

########################
# Apply the FSL math approach
########################
apply_fsl_math_approach(anatomical_path, resulting_mask_path, betted_brain_path)


In [7]:
# Display
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)


(ipykernel_launcher.py:5390): Gdk-WARNING **: 08:50:00.518: gdkdrawable-x11.c:952 drawable is not a pixmap or window


In [ ]:
################
# Segmentation #
################

import os, glob
from fsl.wrappers import fast


# Change the paths if needed!
data_dir = os.getcwd()  
anatomical_path = os.path.join(data_dir, 'T1w.nii.gz')           # Original brain
bet_path = os.path.join(data_dir, 'T1w_brain.nii.gz')            # Skull-stripped brain (output of BET)
segmentation_path = os.path.join(data_dir, 'T1w_fast')           # Output prefix for segmentation results

# Clean up old FAST results
for f in glob.glob(os.path.join(data_dir, '*fast*')):
    os.remove(f)

# Run FAST segmentation on skull-stripped brain
fast_target = bet_path  # BET result is recommended for segmentation
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

print("FAST segmentation complete!")

In [11]:
# Display
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)  # base brain

# Load the three partial volume estimate maps (gray matter, white matter, CSF)
fsleyesDisplay.load(glob.glob(os.path.join(data_dir, '*pve_0*'))[0])  # CSF
fsleyesDisplay.load(glob.glob(os.path.join(data_dir, '*pve_1*'))[0])  # Gray Matter
fsleyesDisplay.load(glob.glob(os.path.join(data_dir, '*pve_2*'))[0])  # White Matter

# Assign colors for clarity
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'    # CSF
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'  # GM
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'   # WM